In [1]:
import copernicusmarine
import xarray as xr
import numpy as np
from dask.distributed import Client
import pandas as pd
from pathlib import Path
import hvplot.xarray

In [2]:
# copernicusmarine.login()

In [3]:
date_start = pd.date_range("1998-01-01", "2023-10-01", freq="1MS")
date_end = pd.date_range("1998-02-01", "2023-11-01", freq="1MS")

export_filepath = Path("./raw/Calcofi_forcing.zarr")

coordinates = {"minimum_latitude": 0, "maximum_latitude": 57, "minimum_longitude": -180, "maximum_longitude": -75}

physic_dataset_name = "cmems_mod_glo_bgc_my_0.083deg-lmtl-Fphy_PT1D-i"
physic_variable = [
    "sea_water_potential_temperature_vertical_mean_over_pelagic_layer",
    "sea_water_pelagic_layer_bottom_depth",
]
bio_dataset_name = "cmems_mod_glo_bgc_my_0.083deg-lmtl_PT1D-i"
bio_variable = [
    "net_primary_productivity_of_biomass_expressed_as_carbon_in_sea_water",
    "mass_content_of_zooplankton_expressed_as_carbon_in_sea_water",
]

latitude = np.arange(0.5, 55.5, 1)
longitude = np.arange(-179.5, -76.5, 1)

In [ ]:
for start, end in zip(date_start, date_end):
    if (not export_filepath.exists()) or (end not in xr.open_dataset(export_filepath, engine="zarr").time):
        coordinates.update({"start_datetime": start.strftime("%Y-%m-%d"), "end_datetime": end.strftime("%Y-%m-%d")})
        print(coordinates)

        physic = copernicusmarine.open_dataset(dataset_id=physic_dataset_name, **coordinates, variables=physic_variable)
        bio = copernicusmarine.open_dataset(dataset_id=bio_dataset_name, **coordinates, variables=bio_variable)

        physic = physic.interp(latitude=latitude, longitude=longitude, method="linear").load()
        bio = bio.interp(latitude=latitude, longitude=longitude, method="linear").load()

        results = xr.merge([physic, bio])

        if export_filepath.exists():
            results = xr.concat([xr.open_dataset(export_filepath, engine="zarr"), results], dim="time")
        results.to_zarr(export_filepath, mode="w")

In [4]:
from dask.distributed import Client

In [5]:
client = Client()

In [6]:
data = xr.open_dataset(
    "../../../../../Data/phd/FORCING/Calcofi_forcing.zarr",
    engine="zarr",
    chunks="auto",
)
data

<xarray.Dataset> Size: 4GB
Dimensions:              (time: 9745, depth: 3, latitude: 55, longitude: 103)
Coordinates:
  * depth                (depth) int32 12B 1 2 3
  * latitude             (latitude) float64 440B 0.5 1.5 2.5 ... 52.5 53.5 54.5
  * longitude            (longitude) float64 824B -179.5 -178.5 ... -78.5 -77.5
  * time                 (time) datetime64[ns] 78kB 1998-01-01 ... 2023-11-01
Data variables:
    T                    (time, depth, latitude, longitude) float64 1GB dask.array<chunksize=(976, 3, 55, 103), meta=np.ndarray>
    npp                  (time, latitude, longitude) float64 442MB dask.array<chunksize=(2960, 55, 103), meta=np.ndarray>
    pelagic_layer_depth  (time, depth, latitude, longitude) float64 1GB dask.array<chunksize=(976, 3, 55, 103), meta=np.ndarray>
    zooc                 (time, latitude, longitude) float64 442MB dask.array<chunksize=(2960, 55, 103), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.7
    history:      Created on 2024-09-6
    institution:  CLS
    references:   http://www.cls.fr; http://www.seapodym.eu
    source:       SEAPODYM-LMTL 3.0.0
    title:        Physical forcings of global ocean low and mid trophic level...

In [7]:
grouped_data = data.groupby("time").mean(dim="time").load()

/Users/adm-lehodey/Documents/Workspace/Projects/Seapopym-optimisation/.venv/lib/python3.12/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 49.85 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


In [8]:
grouped_data

<xarray.Dataset> Size: 3GB
Dimensions:              (time: 9436, depth: 3, latitude: 55, longitude: 103)
Coordinates:
  * depth                (depth) int32 12B 1 2 3
  * latitude             (latitude) float64 440B 0.5 1.5 2.5 ... 52.5 53.5 54.5
  * longitude            (longitude) float64 824B -179.5 -178.5 ... -78.5 -77.5
  * time                 (time) datetime64[ns] 75kB 1998-01-01 ... 2023-11-01
Data variables:
    T                    (time, depth, latitude, longitude) float64 1GB 26.31...
    npp                  (time, latitude, longitude) float64 428MB 262.0 ... nan
    pelagic_layer_depth  (time, depth, latitude, longitude) float64 1GB 110.1...
    zooc                 (time, latitude, longitude) float64 428MB 0.1867 ......
Attributes:
    Conventions:  CF-1.7
    history:      Created on 2024-09-6
    institution:  CLS
    references:   http://www.cls.fr; http://www.seapodym.eu
    source:       SEAPODYM-LMTL 3.0.0
    title:        Physical forcings of global ocean low and mid trophic level...

In [9]:
grouped_data.to_zarr(
    "../../../../../Data/phd/FORCING/Calcofi_forcing_grouped.zarr",
    mode="w",
    consolidated=True,
    compute=True,
)